In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # Force remount to refresh access


Mounted at /content/drive


In [2]:
import os

DATA_PATH = "/content/drive/My Drive/Audio"

# Check if the dataset folder exists
if os.path.exists(DATA_PATH):
    print("✅ Dataset folder found!")
    print("Files inside:", os.listdir(DATA_PATH))
else:
    print("❌ Dataset folder NOT found! Check the path.")


✅ Dataset folder found!
Files inside: ['ambulance', 'traffic', 'police', 'firetruck']


In [3]:
import os

DATA_PATH = "/content/drive/My Drive/Audio"

# List all file paths
file_paths = [os.path.join(DATA_PATH, f) for f in os.listdir(DATA_PATH)]
print("Example file path:", file_paths[0])  # Print one file path


Example file path: /content/drive/My Drive/Audio/ambulance


In [5]:
import os
import librosa
import numpy as np

DATA_PATH = "/content/drive/My Drive/Audio"

X, Y = [], []

# Loop through each folder (class)
for label in os.listdir(DATA_PATH):
    folder_path = os.path.join(DATA_PATH, label)

    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)

            try:
                # Load and extract features
                audio, sr = librosa.load(file_path, sr=22050)
                mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
                mfccs = np.mean(mfccs, axis=1)  # Flatten to 1D

                X.append(mfccs)
                Y.append(label)  # Store the folder name as label

            except Exception as e:
                print(f"Error loading {file_path}: {e}")

X = np.array(X)
Y = np.array(Y)
print("✅ Dataset Loaded! Shape:", X.shape, Y.shape)


✅ Dataset Loaded! Shape: (1675, 40) (1675,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Encode class labels into numbers (0,1,2,3)
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)  # Converts 'ambulance' → 0, 'police' → 1, etc.
Y_categorical = to_categorical(Y_encoded)   # One-hot encoding for CNN

# Split into train & test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_categorical, test_size=0.2, random_state=42)

# Reshape for CNN (Adding a channel dimension)
X_train = X_train[..., np.newaxis]  # Shape: (samples, features, 1)
X_test = X_test[..., np.newaxis]
print("✅ Data ready! Train shape:", X_train.shape, "Test shape:", X_test.shape)


✅ Data ready! Train shape: (1340, 40, 1) Test shape: (335, 40, 1)


In [ ]:
import os
import numpy as np
import librosa

DATA_PATH = "/content/drive/My Drive/ambulance"

X = []
Y = []

# Function to extract MFCC features
def extract_features(file_path):
    try:
        audio, sr = librosa.load(file_path, sr=22050)  # Load audio
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)  # Extract MFCC (40 features)
        return np.mean(mfcc.T, axis=0)  # Take mean over time axis
    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")
        return None

for file in os.listdir(DATA_PATH):
    file_path = os.path.join(DATA_PATH, file)
    if file_path.endswith(".wav"):  # Ensure it's an audio file
        features = extract_features(file_path)
        if features is not None:
            X.append(features)  # Append extracted features
            Y.append(file)  # Append filename as label (modify if needed)

# Convert to NumPy arrays
X = np.array(X)
Y = np.array(Y)

# ✅ Now reshaping X for CNN input
X = X.reshape(X.shape[0], X.shape[1], 1)  # (samples, features, 1)

print(f"✅ Data loaded successfully! X shape: {X.shape}, Y shape: {Y.shape}")


✅ Data loaded successfully! X shape: (400, 40, 1), Y shape: (400,)


In [7]:
from tensorflow.keras import models, layers

# Define CNN model
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(np.unique(Y)), activation='softmax')  # Output neurons = num of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 38, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 19, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 17, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 8, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 156,676 (612.02 KB)

 Trainable params: 156,676 (612.02 KB)

 Non-trainable params: 0 (0.00 B)

None


In [8]:
history = model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_test, Y_test))


Epoch 1/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.5876 - loss: 1.3127 - val_accuracy: 0.9104 - val_loss: 0.2932
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9016 - loss: 0.3343 - val_accuracy: 0.9343 - val_loss: 0.1893
Epoch 3/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9232 - loss: 0.2218 - val_accuracy: 0.9731 - val_loss: 0.1467
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.9463 - loss: 0.1751 - val_accuracy: 0.9582 - val_loss: 0.1273
Epoch 5/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.9568 - loss: 0.1347 - val_accuracy: 0.9552 - val_loss: 0.1268
Epoch 6/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9524 - loss: 0.1261 - val_accuracy: 0.9254 - val_loss: 0.1854
Epoch 7/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9663 - loss: 0.1098 - val_accuracy: 0.9552 - val_loss: 0.0993
Epoch 8/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9702 - loss: 0.1103 - val_accuracy: 0.9672 - v

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define CNN model
model = keras.Sequential([
    layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(np.unique(Y_encoded)), activation='softmax')  # Output layer
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 38, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 19, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 17, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 8, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 400)                 │          51,600 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 207,760 (811.56 KB)

 Trainable params: 207,760 (811.56 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit(X_train, Y_train, epochs=21, batch_size=16, validation_data=(X_test, Y_test))


Epoch 1/21
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9942 - loss: 0.0132 - val_accuracy: 0.9851 - val_loss: 0.0462
Epoch 2/21
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9908 - loss: 0.0199 - val_accuracy: 0.9821 - val_loss: 0.0882
Epoch 3/21
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9890 - loss: 0.0218 - val_accuracy: 0.9612 - val_loss: 0.1572
Epoch 4/21
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9947 - loss: 0.0236 - val_accuracy: 0.9791 - val_loss: 0.0648
Epoch 5/21
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9952 - loss: 0.0142 - val_accuracy: 0.9791 - val_loss: 0.0634
Epoch 6/21
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9896 - loss: 0.0274 - val_accuracy: 0.9791 - val_loss: 0.0826
Epoch 7/21
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9955 - loss: 0.0119 - val_accuracy: 0.9761 - val_loss: 0.0523
Epoch 8/21
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9985 - loss: 0.0111 - val_accuracy: 0.9731 - val_lo

In [11]:
test_loss, test_acc = model.evaluate(X_test, Y_test)
print("Test Accuracy:", round(test_acc * 100, 2), "%")


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9911 - loss: 0.0285     
Test Accuracy: 98.81 %


In [13]:
model.save("siren_detection_model.keras")  # New format



In [14]:
from tensorflow.keras.models import load_model

model = load_model("siren_detection_model.keras")


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
